In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import xgboost as xgb
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report
from sklearn import preprocessing, neighbors
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train.csv',names=['key','V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27','V28','V29','V30','V31','V32','V33','V34','V35','V36','V37','V38','V39','V40','V41','V42','V43','V44','V45','V46','V47','V48','V49','V50','V51','V52','V53','V54','label'])
#train = pd.read_csv('../input/train.csv')
train.set_index('key',inplace=True)
test = pd.read_csv('../input/test.csv',names=['key','V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27','V28','V29','V30','V31','V32','V33','V34','V35','V36','V37','V38','V39','V40','V41','V42','V43','V44','V45','V46','V47','V48','V49','V50','V51','V52','V53','V54'])
keys = test['key']
test.set_index('key',inplace=True)
#train.head().T
#test.describe()

In [ ]:
# Some useful parameters which will come in handy later on
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)

# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
    
# Class to extend XGboost classifer

In [ ]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [ ]:
rf_params = {
    'n_jobs': -1,
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
}

# AdaBoost parameters
ada_params = {
    
}

# Gradient Boosting parameters
gb_params = {
}

# Support Vector Classifier parameters 
svc_params = {

}

# Create 5 objects that represent our 4 models
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
#svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)

In [ ]:
# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['label'].ravel()
train = train.drop(['label'], axis=1)
x_train = train.values # Creates an array of the train data
x_test = test.values # Creats an array of the test data

In [ ]:
# Create our OOF train and test predictions. These base results will be used as new features
et_oof_train, et_oof_test = get_oof(et, x_train, y_train, x_test) # Extra Trees
print("Extra trees done.")
rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test) # Random Forest
print("Random forest done.")
ada_oof_train, ada_oof_test = get_oof(ada, x_train, y_train, x_test) # AdaBoost 
print("Ada boost done.")
gb_oof_train, gb_oof_test = get_oof(gb,x_train, y_train, x_test) # Gradient Boost
print("Gradient boost done.")
#svc_oof_train, svc_oof_test = get_oof(svc,x_train, y_train, x_test) # Support Vector Classifier

print("Training is complete")

In [ ]:
rf_feature = rf.feature_importances(x_train,y_train)
et_feature = et.feature_importances(x_train, y_train)
ada_feature = ada.feature_importances(x_train, y_train)
gb_feature = gb.feature_importances(x_train,y_train)

In [ ]:
rf_features = [2.29218640e-01, 5.07976113e-02, 3.45268299e-02, 6.07001194e-02,
 5.88962786e-02, 1.23801514e-01, 4.05541285e-02, 4.43214848e-02,
 4.26663136e-02, 1.16203955e-01, 1.03467189e-02, 4.83883956e-03,
 1.10285082e-02, 2.15617204e-02, 5.33079539e-04, 5.98740082e-03,
 1.13545246e-03, 5.32129940e-03, 1.96519738e-05, 6.96338842e-04,
 4.91044098e-05, 3.68266888e-05, 1.82071560e-04, 3.20709622e-03,
 4.18900432e-03, 1.93939203e-02, 6.43946687e-03, 1.30178408e-04,
 7.51452553e-11, 9.22836763e-04, 1.16783856e-03, 3.65046008e-04,
 9.81192727e-04, 2.34960309e-03, 8.44450755e-04, 1.33955859e-02,
 7.21112145e-03, 4.77670285e-03, 3.20539043e-04, 7.58097005e-04,
 9.46391963e-04, 3.37589245e-04, 1.26157665e-02, 6.17077222e-03,
 4.59745624e-03, 6.14358450e-03, 5.17679236e-03, 8.04397217e-04,
 6.86039295e-04, 3.67866175e-05, 6.63705389e-05, 1.34657142e-02,
 1.19628155e-02, 7.11295559e-03]
et_features = [2.02558328e-01, 5.47304813e-02, 3.92140576e-02, 6.56113603e-02,
 6.13014560e-02, 1.13233866e-01, 4.52391087e-02, 4.92659480e-02,
 4.53725799e-02, 1.03476455e-01, 1.33060178e-02, 3.41038518e-03,
 1.16080641e-02, 2.50276087e-02, 2.22770505e-04, 5.71476376e-03,
 7.48418953e-04, 6.48405443e-03, 2.92312821e-04, 1.52008434e-03,
 1.01622991e-04, 1.08205057e-04, 3.77631740e-04, 3.09734203e-03,
 4.85255383e-03, 2.37612514e-02, 8.79480168e-03, 3.13807109e-04,
 2.35968056e-08, 1.03799314e-03, 1.01662864e-03, 8.64318609e-04,
 1.64415727e-03, 2.58828622e-03, 1.17149416e-03, 1.71843042e-02,
 1.07995646e-02, 4.40935445e-03, 4.56030304e-04, 1.49068624e-03,
 7.95395660e-04, 7.03595054e-04, 7.09328267e-03, 5.44152646e-03,
 2.99063922e-03, 4.53557871e-03, 4.05038240e-03, 1.48303045e-03,
 1.42507745e-03, 8.97843104e-05, 1.40375140e-04, 1.47458929e-02,
 1.74928791e-02, 6.60438209e-03,]
ada_features = [0.36, 0.  , 0.  , 0.06, 0.02, 0.06, 0.02, 0.06, 0.02, 0.04, 0.  , 0.02, 0.  , 0. ,
 0.02, 0.02, 0.02, 0.04, 0.  , 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.04, 0.  , 0. ,
 0.  , 0.  , 0.02, 0.02, 0.  , 0.  , 0.02, 0.02, 0.02, 0.  , 0.  , 0.  , 0.  , 0. ,
 0.02, 0.  , 0.  , 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.02, 0.  ]
gb_features = [3.16518586e-01, 1.55299478e-02, 2.22861667e-04, 7.62268577e-02,
 1.38454699e-02, 1.20133008e-01, 8.38653816e-03, 5.17010171e-02,
 3.66628456e-03, 7.61882799e-02, 2.36133390e-02, 1.69240724e-02,
 1.40646274e-02, 0.00000000e+00, 0.00000000e+00, 3.10605869e-02,
 0.00000000e+00, 3.42351898e-02, 0.00000000e+00, 0.00000000e+00,
 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
 0.00000000e+00, 2.23741571e-02, 2.59521344e-03, 0.00000000e+00,
 0.00000000e+00, 0.00000000e+00, 1.34848210e-02, 0.00000000e+00,
 0.00000000e+00, 7.82530779e-03, 7.76355290e-03, 2.69460268e-02,
 2.54295116e-02, 0.00000000e+00, 1.94220415e-03, 0.00000000e+00,
 2.43475182e-03, 0.00000000e+00, 1.52751191e-02, 6.04548681e-03,
 1.06630852e-02, 1.59077264e-02, 9.70128724e-03, 8.41135958e-03,
 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 7.61497903e-03,
 1.32687440e-02, 0.00000000e+00]

In [ ]:
cols = train.columns.values
# Create a dataframe with features
feature_dataframe = pd.DataFrame( {'features': cols,
     'Random Forest feature importances': rf_features,
     'Extra Trees  feature importances': et_features,
      'AdaBoost feature importances': ada_features,
    'Gradient Boost feature importances': gb_features
    })

In [ ]:
# Create the new column containing the average of values

feature_dataframe['mean'] = feature_dataframe.mean(axis= 1) # axis = 1 computes the mean row-wise
feature_dataframe.head(3)

In [ ]:
base_predictions_train = pd.DataFrame( {'RandomForest': rf_oof_train.ravel(),
     'ExtraTrees': et_oof_train.ravel(),
     'AdaBoost': ada_oof_train.ravel(),
      'GradientBoost': gb_oof_train.ravel()
    })
base_predictions_train.head()

In [ ]:
x_train = np.concatenate(( et_oof_train, rf_oof_train, ada_oof_train, gb_oof_train), axis=1)
x_test = np.concatenate(( et_oof_test, rf_oof_test, ada_oof_test, gb_oof_test), axis=1)

In [ ]:
gbm = xgb.XGBClassifier(
    #learning_rate = 0.02,
 n_estimators= 2000,
 max_depth= 4,
 min_child_weight= 2,
 #gamma=1,
 gamma=0.9,                        
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1).fit(x_train, y_train)
predictions = gbm.predict_proba(x_test)

In [ ]:
# Generate Submission File 
print(predictions[:,0])
StackingSubmission = pd.DataFrame({ 'key': keys,
                            'score': predictions[:,1] })
StackingSubmission.to_csv("StackingSubmission.csv",index = False)